# Analysis of the Coronawiki dataset

## Preliminary analysis

### Load the different files and study each dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd
import json
import gzip
 


path = 'data/'

#Load csv files
df_interventions = pd.read_csv(path + 'interventions.csv')
df_google_mobility = pd.read_csv(path + 'Global_Mobility_Report.csv.gz')
df_apple_mobility = pd.read_csv(path + 'applemobilitytrends-2020-04-20.csv')
df_topics_linked = pd.read_csv(path + 'topics_linked.csv.xz')
df_democracy = pd.read_csv(path + 'democracy.csv')

#Load the (gzipped) json file
with gzip.open(path + 'aggregated_timeseries.json.gz', "rb") as f:
    json_timeseries = json.loads(f.read().decode("ascii"))



ModuleNotFoundError: No module named 'geopandas'

### Democracy Dataframe

In [ ]:
#Setting the Years after 1960 as indexes in order to plot the evolution of the democracy index 
df_democracy_plot=df_democracy[df_democracy['Year']>=1960]
df_democracy_plot.set_index('Year',inplace=True)

Plot the evolution of the democracy index for a selection of countries

In [ ]:
#Grouping the countries and ploting their evolution.
df_democracy_plot[(df_democracy_plot['Entity'].isin(['Denmark','Finland','Germany','Italy','Japan','Netherlands','Norway','Serbia','South Korea', 'Spain', 'Sweden', 'Turkey']))].groupby('Entity')['libdem_vdem_owid'].plot(x='Year',y='libdem_vdem_owid',legend=True)

#Drawing the limit of the year 2019 since this is the year where we will consider the democracy score for the countries for our study
plt.axvline(x = 2020, color = 'r',ls='--', label = 'Year = 2020')
plt.title('Evolution of the Democracy Index')
plt.legend(bbox_to_anchor=(1.0, 1.0))
plt.show()
#resent the index from the Year to before the plotting...
df_democracy_plot.reset_index(inplace=True)

Plot the 2020 democracy index on a map for the world, Europe and East Asia

In [ ]:
#import the geopandas database in order to plot the maps
countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [ ]:

#change the name of the countries in order to correctly merge the two dataframe.
df_democracy=df_democracy.replace("United States", "United States of America")
df_democracy=df_democracy.replace("Democratic Republic of Congo", "Dem. Rep. Congo")
df_democracy=df_democracy.replace("Central African Republic","Central African Rep.")
df_democracy=df_democracy.replace("Cote d'Ivoire","Côte d'Ivoire")


#merge with the democracy index database in order to associate to each of our wanted country a democracy score.
countries = countries.merge(df_democracy[df_democracy["Year"]==2020][['Entity','libdem_vdem_owid']],left_on='name',right_on='Entity',how='left')
countries.drop('Entity', inplace=True, axis=1)

In [ ]:
#Plotting and drawing the maps with the democracy scores.

ax1=countries.plot(column='libdem_vdem_owid', legend=True,legend_kwds={'label': "Democracy Index",
                        'orientation': "vertical"}, missing_kwds={'color': 'lightgrey'})
ax2=countries.plot(column='libdem_vdem_owid', legend=True,legend_kwds={'label': "Democracy Index",
                        'orientation': "vertical"}, missing_kwds={'color': 'lightgrey'})
ax3=countries.plot(column='libdem_vdem_owid', legend=True,legend_kwds={'label': "Democracy Index",
                        'orientation': "horizontal"}, missing_kwds={'color': 'lightgrey'},figsize=(20, 10))
ax1.set_xlim(-20,40)
ax1.set_ylim(30,75)
ax1.set_axis_off()

ax2.set_xlim(80,150)
ax2.set_ylim(0,45)
ax2.set_axis_off()

ax3.set_ylim(-70,90)
ax3.set_axis_off()

plt.show()



### Interventions dataframe

In [ ]:
#Dataframe overview

df_interventions.head(20)

In [ ]:
#Countries considered in this dataset

print (df_interventions.shape[0], 'countries considered :')

for ctry in df_interventions['lang'] :
    print (ctry, end='  ')

### Apple mobility report

In [ ]:
#Dataframe overview

df_apple_mobility.head(3)

In [ ]:
#Min and max dates of this dataset

apple_cols = df_apple_mobility.columns
print ('Reference day :', apple_cols[3])
print ('Last day :', apple_cols[-1])

In [ ]:
#Regions considered

#Number of cities and country/region considered
print (df_apple_mobility['geo_type'].value_counts(), '\n')

#Regions considered
apple_regions = df_apple_mobility[df_apple_mobility['geo_type']=='country/region']['region']
print ('Regions : \n', apple_regions.unique(), '\n')

#Count the regions where the data is for walking, driving and transit
occurence_counting = apple_regions.value_counts()
print('Nb of regions with data for walking, driving and transit :', occurence_counting[occurence_counting==3].shape[0])
print('Nb of regions with data for only walking and driving :', occurence_counting[occurence_counting==2].shape[0], '\n')

#Cities considered
apple_cities = df_apple_mobility[df_apple_mobility['geo_type']=='city']['region']
print ('Cities : \n', apple_cities.unique(), '\n')

#Count the regions where the data is for walking, driving and transit
occurence_counting = apple_cities.value_counts()
print('Nb of cities with data for walking, driving and transit :', occurence_counting[occurence_counting==3].shape[0])
print('Nb of cities with data for only walking and driving :', occurence_counting[occurence_counting==2].shape[0], '\n')


In [ ]:
#Exemple : plot walking evolution in France

import matplotlib.pyplot as plt

#Extract walking data for France
france_walking = df_apple_mobility[(df_apple_mobility['region']=='France') & (df_apple_mobility['transportation_type']=='walking')]
france_walking.drop(['geo_type', 'region', 'transportation_type'], inplace=True, axis=1)

#Transpose it to have a pandas series and plot it
france_walking = france_walking.transpose()
fig = plt.figure(figsize=(15, 5))
plt.plot(france_walking)
plt.xticks(fontsize=7, rotation=90)
plt.title('Walking Evolution in France')
plt.xlabel('Date')
plt.ylabel('Relative evolution (%)')
plt.show()

### Google mobility report

In [ ]:
#Overview of the dataset

df_google_mobility.head(3)

In [ ]:
#Countries for which we have data
google_countries = df_google_mobility['country_region'].unique()
print ('Data for', google_countries.size, 'countries :')
print (google_countries)

In [ ]:
#The data can be at a more specific scale than the whole country
#For instance in France, the different regions are considered

google_mob_france = df_google_mobility[df_google_mobility['country_region'] == 'France']
print ('Regions considered in France')
print (google_mob_france['sub_region_1'].unique(), '\n')

#Let's now focus on Occitanie
print ('Subregions in Occitanie :')
google_mob_occ = google_mob_france[google_mob_france['sub_region_1'] == 'Occitanie']
print (google_mob_occ['sub_region_2'].unique())

#In occitanie let's focus on Haute-Garonne
google_mob_hg = google_mob_occ[google_mob_france['sub_region_2'] == 'Haute-Garonne']

In [ ]:
#Now let's see what data can be extracted in Haute-Garonne
print ('Columns of the dataframe :')
print (google_mob_hg.columns, '\n')

#Focus on the date
print ('Date range : ')
print ('from', google_mob_hg['date'].iloc[0], 'to', google_mob_hg['date'].iloc[-1], '\n')

#Specific places for which we have data
print ('Types of places considered :')
for col in google_mob_hg.columns :
    if '_percent_change_from_baseline' in col :
        print(' -',col.replace('_percent_change_from_baseline', ''))


### Analysis of the dataframe topics_linked

In [ ]:
#print the shape and head of the dataframe

print ('Shape of the dataframe :', df_topics_linked.shape)
df_topics_linked.head()

In [ ]:
#Print the Wikipedia pages related to a specific topic

#Example for east Asia
topic = 'Geography.Regions.Asia.East Asia'
df_specific_topic = df_topics_linked[df_topics_linked[topic]==True]
print ('The', df_specific_topic.shape[0], 'Wikipedia pages related to', topic, 'are : \n')
print (df_specific_topic['index'])

### Analysis of the dictionary json_timeseries

In [ ]:
# The coronawiki time series data are contained in the dictionary json_timeseries, let's see what is the structure of this dictionary
versions = list(json_timeseries.keys())
versions.sort()

print('The different versions of Wikipedia that are considered are:')
print(versions)

print('We have the visit history on '+str(int(len(versions)/2))+' different versions of Wikipedia, for each of these versions there are two keys one for the normal version and one for the mobile version .m ')

In [ ]:
# Now let's see what is the structure inside one of these version, the german one for example
german = json_timeseries['de']
print('The keys for the german version, and for all other versions are:')
print(list(german.keys()))

In [ ]:
# The key len gives the number of Wikipedia pages for this version
print('key len: The german version of Wikipedia contains '+str(int(german['len']))+' pages \n')

In [ ]:
# The key sum gives the number of daily visits to all pages between 01-01-2018 and 07-31-2020
total_visits = german['sum']
print('key sum: The daily number of visits for these pages during the ten first days of 2018 are:')
print(list(total_visits.values())[:10])
print('\n')

In [ ]:
# Now let's see what is inside the key covid
covid_visits = german['covid']
print('The key covid contains 3 sub-keys: ')
print(list(covid_visits.keys()))
# First, the number of pages that are considered to be linked with COVID-19 is in the key len
print('key len: ' + str(covid_visits['len'])+' pages are linked with COVID-19')
# Then, the key sum gives the number of daily visits to pages linked with COVID-19 between 01-01-2018 and 07-31-2020
print('key sum: The daily number of visits for COVID-19 related pages during the ten first days of 2018 are:')
print(list(covid_visits['sum'].values())[:10])
# Finally, the key percent gives the proportion of COVID-related visits to total visits
print('key percent: The proportion of COVID-related visits to total visits during the ten first days of 2018 are:')
print(list(covid_visits['percent'].values())[:10])
print('\n')

In [ ]:
# Finally the key topics contains a sub-key for each topic that are detailed in the dataframe df_topics_linked.
print('key topics: '+str(len(german['topics'].keys())) + ' topics are considered in this version')
# Inside one of these topics the structure is the same as in the key covid:
print('Structure of a topic: ') 
print(list(german['topics']['Culture.Biography.Biography*'].keys()))
print('\n')
# First, the number of pages that are considered to be linked with this topic is in the key len
print('key len: ' + str(german['topics']['Culture.Biography.Biography*']['len'])+' pages are linked with biography \n')
# Then, the key sum gives the number of daily visits to pages linked with this topic between 01-01-2018 and 07-31-2020
print('key sum: The daily number of visits for pages related to biography during the ten first days of 2018 are:')
print(list(german['topics']['Culture.Biography.Biography*']['sum'].values())[:10])
print('\n')
# Finally, the key percent gives the proportion of visits related to this topic to total visits
print('key percent: The proportion of visits related to biography to total visits during the ten first days of 2018 are:')
print(list(german['topics']['Culture.Biography.Biography*']['percent'].values())[:10])

The structure of the dictionary json_timeseries can be shown with the following graph:

In [ ]:
from IPython.display import Image
Image("json.png")

# First analysis of the evolution of interests

### Analysis of the total number of wikipedia searches

In [ ]:
# a list of the names of the wikipedia versions.
wiki_versions =['sv','de','it','sr','no','ko','da','ja','nl','fi','ca','tr']


#Here we want to find the dat eof the beginning of the lockdown and insert it in the dictionnary in order to later plot it
lockdown_start_dic={'sv':0,'de':0,'it':0,'sr':0,'no':0,'ko':0,'da':0,'ja':0,'nl':0,'fi':0,'ca':0,'tr':0}


def find_lockdown_start(version):
    # Let's put in the list time the list of the days of the considered period
    time = list(json_timeseries[version]['sum'].keys())
    # We just remove the hours
    time = [t[:10] for t in time]

    # Thanks to the dataframe df_interventions we can get the dates of the lockdown in the associated country
    lockdown_start = df_interventions.loc[df_interventions['lang'] == version]['Mobility'].to_list()
    # We compute the index of these dates in the list of the days
    start_index = time.index(lockdown_start[0])-365
    return lockdown_start[0],start_index

#compute the start lockdown indexes and store them in the dictionnary    
for version in wiki_versions:
    lockdown_start_dic[version]=find_lockdown_start(version)[1]
    
print(lockdown_start_dic)

In [ ]:
#This is a function that gives us the x and y axis corresponding to the time stamps starting the year 2019 for the x-axis,
# and the number of searches per days for the y-axis

def wiki_searches_axis(version):
    V=json_timeseries[version]['sum']
    myList = V.items()
    myList = sorted(myList) 
    x, y = zip(*myList) 
    return np.array(x[365:]),np.array(y[365:])

# This function calculates the rolling average of an array x over a window of width w
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


In [ ]:
def plot_wiki_searches(version):

    # Let's put in the list time the list of the days of the considered period
    time = list(json_timeseries[version]['sum'].keys())
    # We just remove the hours
    time = [t[:10] for t in time]

    # For the purpose of plotting our results we create a list of days with only 1 day in 21 and we also create a list of index for these days
    time_reduced = [time[21*i+365] for i in range(int(len(time[365:])/21)+1)]
    time_pos = [21*i for i in range(int(len(time[365:])/21)+1)]


    lockdown_start,start_index = find_lockdown_start(version)
    # We compute the index of these dates in the list of the days
    start_index = time.index(lockdown_start)-365
    time_pos.append(start_index)
    time_reduced.append(lockdown_start+' Start')

    # We also remove two points which are too close from the start/end of the lockdown to have a nice plot
    time_pos.pop(time_reduced.index('2020-03-17'))
    time_reduced.pop(time_reduced.index('2020-03-17'))
    
    #finding our axis
    x,y=wiki_searches_axis(version)

    fig, ax1 = plt.subplots(figsize=(15,5))
    fig.suptitle("Evolution of Wikipedia searches for {version}".format(version=version))

    #compute the rolling mean
    rolling_mean=np.array(moving_average(y,7))
    
    #Plotting everything
    ax1.plot(x, y, alpha=0.3)
    ax1.plot(x[:len(x)-6],rolling_mean)
    ax1.axvline(x = lockdown_start_dic[version], color = 'r',ls='--', label = 'Year = 2020')
    
    
    ax1.set_xticks(time_pos,time_reduced,fontsize=10)
    ax1.tick_params('x',labelrotation=90)
    plt.show()

for version in wiki_versions:
    plot_wiki_searches(version)



In [ ]:

#dictionnary with the population of each country
country_population={'sv':10.28e6,'de':83.09e6,'it':59.73e6,'sr':6.945e6,'no':5.348e6,'ko':51.76e6,'da':5.814e6,'ja':126.6e6,'nl':17.34e6,'fi':5.522e6,'ca':7.566e6,'tr':83.43e6}

#plot the mean on the wikipedia searches for the countries, weighted according to the country's population.
def plot_sum_wiki():
    sum_searches=np.zeros(578)
    
    # Let's put in the list time the list of the days of the considered period
    time = list(json_timeseries['it']['sum'].keys())
    # We just remove the hours
    time = [t[:10] for t in time]

    # For the purpose of plotting our results we create a list of days with only 1 day in 21 and we also create a list of index for these days
    time_reduced = [time[21*i+365] for i in range(int(len(time[365:])/21)+1)]
    time_pos = [21*i for i in range(int(len(time[365:])/21)+1)]
    
    #compute the mean value of the searches
    for K in wiki_versions:
        x,y=wiki_searches_axis(K)
        sum_searches=np.add(sum_searches,country_population[K]*y)
    sum_searches=sum_searches/sum(country_population.values())

    #compute the rolling mean
    rolling_mean=np.array(moving_average(sum_searches,7))
    

    # plotting
    fig, ax1 = plt.subplots(figsize=(15,5))
    fig.suptitle("Evolution of Wikipedia searches for all countries")
    ax1.plot(x, sum_searches, alpha=0.3)
    ax1.plot(x[:len(x)-6],rolling_mean)
    
    #setting the ticks
    ax1.set_xticks(time_pos,time_reduced,fontsize=10)
    ax1.tick_params('x',labelrotation=90)
    plt.show()

plot_sum_wiki()


In [ ]:
#Here we want to plot how each topic evolved during the pandemics with respect to the normal

# Let's put in the list time the list of the days of the considered period
time = list(json_timeseries['it']['topics']['Culture.Sports']['sum'].keys())
# We just remove the hours
time = [t[:10] for t in time]

# Thanks to the dataframe df_interventions we can get the dates of the lockdown in Italy
lockdown_start = df_interventions.loc[df_interventions['lang'] == 'it']['Lockdown'].to_list()
lockdown_end = df_interventions.loc[df_interventions['lang'] == 'it']['Normalcy'].to_list()

# We compute the index of these dates in the list of the days
start_index = time.index(lockdown_start[0])
end_index = time.index(lockdown_end[0])

#Create an empty dataframe which will contain the relative evolution for each topic
df = pd.DataFrame({'topic' : [], 'evolution' : []})

#For each topic, compute its evolution and add it to the dataframe
for topic in json_timeseries['it']['topics'].keys() :
    topic_m = pd.Series(list(json_timeseries['it.m']['topics'][topic]['percent'].values()))
    topic_normal = pd.Series(list(json_timeseries['it']['topics'][topic]['percent'].values()))
    topic_all = topic_normal.add(topic_m)

    #Relative evolution : ratio between the relative topic interset during the pandemic vs before pandemic
    relative_evolution = topic_all[start_index:end_index].mean() / topic_all.mean()

    #Select only the topics where the evolution is strong
    threshold=0.0 #can be increased if you want only the topics which have change above the threshold
    if relative_evolution > 1+threshold or relative_evolution < 1-threshold:
        new_entry = [topic, (relative_evolution-1)*100]
        df.loc[df.shape[0]] = new_entry 

#Sort and plot the dataframe
df.sort_values(by=['evolution'], inplace=True)
plt.figure(figsize=(10, 15))
plt.xlabel('Relative evolution (%)')
plt.barh(df['topic'], df['evolution'])

On this plot, the observed trend seems to be that italians' interest during lockdown decreased overall for things that were forbidden (travelling and doing sports), and increased overall for things that were allowed (watching films and reading books). The Italians thus seem to have conformed to lockdown rather than to be interested in what was no longer accessible.
Nevertheless, there are some exceptions like the categories Southern and North Africa that have increased, or the category Music that has decreased.

To develop this work we should do the same kind of analysis for different countries, especially in countries where the lockdown conditions were different.


In [ ]:
# Let's plot the evolution of the relative number of visits for two topics that have a large variation: Sports and Books

# For books and sports we create a list with the daily relative number of visits on the italian version linked to these topics
books = pd.Series(list(json_timeseries['it']['topics']['Culture.Media.Books']['percent'].values()))
sport = pd.Series(list(json_timeseries['it']['topics']['Culture.Sports']['percent'].values()))

# We do the same on the mobile version of the italian Wikipedia
books_m = pd.Series(list(json_timeseries['it.m']['topics']['Culture.Media.Books']['percent'].values()))
sport_m = pd.Series(list(json_timeseries['it']['topics']['Culture.Sports']['percent'].values()))

# For these two topics we add the values for the mobile and classic version
books_tot = books.add(books_m)
sport_tot = sport.add(sport_m)

# For the purpose of plotting our results we create a list of days with only 1 day in 21 and we also create a list of index for these days
time_reduced = [time[21*i] for i in range(int(len(time)/21)+1)]
time_pos = [21*i for i in range(int(len(time)/21)+1)]

# We add to the reduced list of days and the list of their index the dates of lockdowns in Italy
time_pos.append(start_index)
time_reduced.append(lockdown_start[0]+' Start')

time_pos.append(end_index)
time_reduced.append(lockdown_end[0]+' End')

# We also remove two points which are too close from the start/end of the lockdown to have a nice plot
time_pos.pop(time_reduced.index('2020-03-09'))
time_reduced.pop(time_reduced.index('2020-03-09'))

time_pos.pop(time_reduced.index('2020-06-22'))
time_reduced.pop(time_reduced.index('2020-06-22'))

fig, [ax1,ax2] = plt.subplots(2,1,figsize=(20,20),sharex=True)
ax1.plot(books_tot.rolling(7).mean(),label='Books')
ax2.plot(sport_tot.rolling(7).mean(),color='green',label='Sports')
ax2.set_xticks(time_pos,time_reduced,fontsize=10)
ax2.tick_params('x',labelrotation=90)
ax1.axvline(x=start_index)
ax1.axvline(x=end_index)
ax2.axvline(x=start_index)
ax2.axvline(x=end_index)
fig.suptitle('Evolution of the rolling mean over 7 days of the relative number \n of visits linked with Books and Sports on the italian Wikipedia',fontsize=20)

ax1.legend(fontsize=15)
ax2.legend(fontsize=15)

These two graphs confirm the previous results and give a better view of the overall evolution of interest in sport and books in Italy. Interest in books peaks (over the period considered) about a week after the start of the lockdown and remains higher overall during the lockdown than over the rest of the period. It is also clearly visible that this interest starts to decrease after several weeks of lockdown and eventually drops when the situation returns to normal. 

On the contrary, interest in sport reaches its lowest level about a week after lockdown, increases slowly over the weeks and then very quickly when returning to normal.

In [ ]:
# Like the previous bar plot, the objective is here to be able to visualize the interest evolution of all the subjects but for each country and on the same figure, in a simple and clear way
# For this we will build a matrix whose color gradient will correspond to the interest evolution of a certain subject in a certain country
import datetime
import time
from datetime import date
import numpy as np
import matplotlib

# First we create a dataframe df_p4 with the data we need for this final part, the column versions contains the two letters corresponding the versions of Wikipedia that are considered, the column dem_index contains the democracy index of the corresponding countries and the column mobility shift contains the relative mobility shift computed previously.
versions = ['tr', 'sr', 'ja', 'it', 'ko', 'ca', 'nl', 'de', 'fi', 'no', 'sv', 'da']
dem_index = [0.11, 0.27, 0.74, 0.78, 0.79, 0.8, 0.81, 0.83, 0.83, 0.86, 0.88, 0.88]
mobility_shift = [-0.777, -0.785, -0.514, -0.904, -0.640 , -0.943, -0.657, -0.620, -0.416, -0.544, -0.418, -0.525]
dict = {'country': versions, 'democracy index': dem_index, 'mobility shift': mobility_shift}
df_p4 = pd.DataFrame(data = dict)

# To facilitate the building of the final matrix we sort the dataframe either with the democracy index or with the mobility shift
sort = 'mobility shift' # can be changed with 'dem_index' if we want the output as the function of the democracy index
df_p4 = df_p4.sort_values(sort) 
df_p4 = df_p4.reset_index(drop = True)

# Now we build a list period with all the days (with the datetime type) of the considered period
period = []
d0 = date(2018,1,1) # Day of start
d1 = date(2020,7,31) # Day of end
delta = datetime.timedelta(days=1) # Duration between two points: one day

while d0 <= d1:
    period.append(d0.strftime('%Y-%m-%d'))
    d0 += delta 

# Thanks to the dataframe df_interventions we can get the dates of the lockdown in every countries that we put in a new dataframe lockdown
lockdown = df_interventions.loc[df_interventions['lang'].isin(df_p4['country'])][['lang','Mobility','Normalcy']]

# There are two missing values: the return to normalcy in Catalonia and Turkey. We will assume that the return to normalcy is 2020-07-01 for these countries
lockdown.iloc[8,2] = '2020-07-01'
lockdown.iloc[11,2] = '2020-07-01'

# We compute the index of these dates in the list of the days and we put them in two new columns 'start_index' and 'end_index'
lockdown['start_index'] = [period.index(date) for date in lockdown['Mobility']]
lockdown['end_index'] = [period.index(date) for date in lockdown['Normalcy']]

# We create a dataframe where each row is characterized by a country in the column 'country', a topic in the column 'topic' and the relative evolution of interest for this topic in this country
df_evolution = pd.DataFrame({'country' : [], 'topic' : [], 'evolution' : []})
 
#For each topic and each country, we compute its evolution and add it to the dataframe
for d in df_p4['country']:
    for topic in json_timeseries[d]['topics'].keys() :
        # We get the daily data of the share of this topic in the global wikipedia searches in this country (mobile and desktop version)
        topic_percent_mobile = pd.Series(list(json_timeseries[d + '.m']['topics'][topic]['percent'].values()))
        topic_percent_desktop = pd.Series(list(json_timeseries[d]['topics'][topic]['percent'].values()))

        # We get the daily data of wikipedia visits related to this topic in this country (mobile and desktop version)
        topic_total_mobile = pd.Series(list(json_timeseries[d + '.m']['topics'][topic]['sum'].values()))
        topic_total_desktop = pd.Series(list(json_timeseries[d]['topics'][topic]['sum'].values()))

        # To know the proportion that this subject represents in the wikipedia visits we combine the data of the mobile version and the desktop version by making a weighted average of the two versions
        topic_all = (topic_percent_mobile * topic_total_mobile + topic_percent_desktop * topic_total_desktop) / (topic_total_mobile + topic_total_desktop)

        # We compute the relative evolution which is defined as the ratio between the average topic interest during the pandemic vs the average topic interest during the saeme period but one year before

        # Index of the day of the start and end of the lockdown in the list of the days
        start_index = lockdown.loc[lockdown['lang'] == d]['start_index'].tolist()[0]
        end_index = lockdown.loc[lockdown['lang'] == d]['end_index'].tolist()[0]

        relative_evolution = (topic_all[start_index:end_index].mean() -  topic_all[start_index - 365 : end_index - 365].mean()) / topic_all[start_index - 365 : end_index - 365].mean()

        # We create a new entry for the output dataframe with all the data needed
        new_entry = [d, topic, relative_evolution*100]
        df_evolution.loc[df_evolution.shape[0]] = new_entry 

# Now let's build the final matrix

# The number of rows in the number of different topics in the dataframe df_evolution
topics = list(set(df_evolution['topic'].tolist()))
nb_rows = len(topics)

# The number of columns is the number of countries
countries = df_p4['country']
nb_columns = len(countries)

# We create a matrix full of zeros with an additional column that will contain the average of all others columns
A = np.zeros((nb_rows, nb_columns + 1))

# Let's fill this matrix
for i in range(nb_rows):
    for j in range(nb_columns):
        # For each element of the matrix we retrieve the corresponding data in the DataFrame built previously
        A[i,j] = df_evolution.loc[(df_evolution['country'] == countries[j]) & (df_evolution['topic'] == topics[i])]['evolution'].tolist()[0]
    # The last column is the average evolution of the corresponding subject over all countries
    A[i,nb_columns] = sum([A[i,j] for j in range(nb_columns)]) / nb_columns

# Let's remove some topics whose evolution is not significant: if the average evolution is less than 10 we remove the corresponding row
# The threshold can be modified
removed_rows = []
for i in range(nb_rows):
    if abs(A[i, nb_columns]) <= 10:
        removed_rows.append(i)

A = np.delete(A, removed_rows, axis = 0)

# We also remove the corresponding topics in the list of the topics
topics = np.delete(topics, removed_rows)

# Then we sort the matrix (and the list of the topics) according to the value of the last column
l = A[:, nb_columns].argsort()

A = A[l]
topics = np.array(topics)[l]

# We can delete the last column before plotting the result
A = np.delete(A, (nb_columns), axis = 1)

# Let's plot our result
fig = plt.figure(figsize=(30,15))
ax = fig.add_subplot(111)

'''
Function to offset the "center" of a colormap. Useful for data with a negative min and positive max and you want the middle of the colormap's dynamic range to be at zero.

Input
-----
    cmap : The matplotlib colormap to be altered
    start : Offset from lowest point in the colormap's range.
        Defaults to 0.0 (no lower offset). Should be between 0.0 and `midpoint`.
    midpoint : The new center of the colormap. Defaults to 0.5 (no shift). Should be between 0.0 and 1.0. In general, this should be  1 - vmax / (vmax + abs(vmin))
        For example if your data range from -15.0 to +5.0 and you want the center of the colormap at 0.0, `midpoint` should be set to  1 - 5/(5 + 15)) or 0.75
    stop : Offset from highest point in the colormap's range.
        Defaults to 1.0 (no upper offset). Should be between `midpoint` and 1.0.
'''
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    #plt.register_cmap(cmap=newcmap)

    return newcmap

mp = 1 - A[:, :nb_columns].max() / (A[:, :nb_columns].max() + abs(A[:, :nb_columns].min()))

new_cmap = shiftedColorMap(matplotlib.cm.seismic, midpoint=mp)

cax = ax.matshow(A, interpolation='nearest', cmap = new_cmap)
fig.colorbar(cax)

xaxis = np.arange(len(countries))
yaxis = np.arange(len(topics))
ax.set_xticks(xaxis)
ax.set_yticks(yaxis)
ax.set_xticklabels(countries)
ax.set_yticklabels(topics)

plt.show()

In [ ]:
import seaborn as sns
# Now we will focus on the last two lines of the matrix which correspond to video games and movies

# We will do a plot where each point represents a version of wikipedia, with the mobility shift in the associated country on the x-axis and the gain in interest in films and video games on the y-axis (weighted average of the two subjects).

# The x-axis will be the mobility shift
X = df_p4['mobility shift']*100

# In these two list we will put the total number of visits linked with films or video games in each country. This need to compute the weighted average in the next step.

films_total = []
videogames_total = []

for d in df_p4['country']:
    films_total.append(pd.Series(list(json_timeseries[d]['topics']['Culture.Media.Films']['sum'].values())).mean() + pd.Series(list(json_timeseries[d + '.m']['topics']['Culture.Media.Films']['sum'].values())).mean())
    videogames_total.append(pd.Series(list(json_timeseries[d]['topics']['Culture.Media.Video games']['sum'].values())).mean() + pd.Series(list(json_timeseries[d + '.m']['topics']['Culture.Media.Video games']['sum'].values())).mean())

films_total = np.array(films_total)
videogames_total = np.array(videogames_total)

# The y-axis is the average of the gain in interest in videogames and films weighted by the number of visits for these two topics
Y = (np.array(A[16, :]) * films_total + np.array(A[17,:]) * videogames_total) / (films_total + videogames_total)

colors = sns.color_palette('Set2')
plt.figure(figsize=(15,7))
plt.scatter(X, Y, color = colors[0], label = 'Films and Video Games')
plt.xlabel('Mobility shift (%)')
plt.ylabel('Evolution of the interest (%)')

for i, txt in enumerate(df_p4['country']):
    plt.annotate(txt, (X[i], Y[i]))

plt.legend()
plt.show()


In [ ]:
# Now we can do a linear regression to approximate the effect of the mobility shift in the interest in films and video games

import statsmodels.formula.api as smf
d = {'mobility_shift': X, 'interest': Y}
df_reglin = pd.DataFrame(data = d)

np.random.seed(2)

mod = smf.ols(formula='interest ~ mobility_shift', data=df_reglin)

res = mod.fit()

print(res.summary())

plt.figure(figsize=(15,7))
plt.scatter(X, Y, color = colors[0], label = 'Films and Video Games')
plt.plot(np.linspace(-100,-40,3), [-8.1772 - 0.5804 * k for k in np.linspace(-100,-40,3)], color = colors[1])
plt.text(-80, -8.1772 - 0.5804 * (-80), 'y = -8.1772 - 0.5804 * x', color = colors[1])
plt.xlabel('Mobility shift (%)')
plt.ylabel('Evolution of the interest (%)')

for i, txt in enumerate(df_p4['country']):
    plt.annotate(txt, (X[i], Y[i]))

plt.legend()
plt.show()

## Analysis of the change in mobility during the covid pandemic

### Plotting the mobility evolution in a selection of countries

In [ ]:
#Exemple : plot walking evolution in France

import matplotlib.pyplot as plt

#Getting sns colors
colors = sns.color_palette("rocket")

#Extract walking data for France
france_walking = df_apple_mobility[(df_apple_mobility['region']=='France') & (df_apple_mobility['transportation_type']=='walking')]
france_walking.drop(['geo_type', 'region', 'transportation_type'], inplace=True, axis=1)

#Extract data for Mexico
mexico_walking = df_apple_mobility[(df_apple_mobility['region']=='Mexico') & (df_apple_mobility['transportation_type']=='walking')]
mexico_walking.drop(['geo_type', 'region', 'transportation_type'], inplace=True, axis=1)


#Extract data for Vietnam
vietnam_walking = df_apple_mobility[(df_apple_mobility['region']=='Vietnam') & (df_apple_mobility['transportation_type']=='walking')]
vietnam_walking.drop(['geo_type', 'region', 'transportation_type'], inplace=True, axis=1)

#Transpose it to have a pandas series and plot it
france_walking = france_walking.transpose()
mexico_walking = mexico_walking.transpose()
vietnam_walking = vietnam_walking.transpose()
fig = plt.figure(figsize=(15, 5))
plt.plot(france_walking.rolling(7, center=True).mean(), color=colors[2], label='France (democracy index = 0.798)')
plt.plot(mexico_walking.rolling(7, center=True).mean(), color = colors[3], label='Mexico (democracy index = 0.412)')
plt.plot(vietnam_walking.rolling(7, center=True).mean(), color = colors[4], label='Vietnam (democracy index = 0.112)')
plt.plot(france_walking, alpha=.2, color=colors[0])
plt.plot(mexico_walking, alpha=.2, color=colors[1])
plt.plot(vietnam_walking, alpha=.2, color=colors[2])
plt.xticks(fontsize=7, rotation=90)
plt.title('Walking Evolution in France, Mexico and Vietnam')
plt.xlabel('Date')
plt.ylabel('Relative evolution (%)')
plt.legend()
#plt.savefig('images/walking_evolution_fr_me_viet.png')
plt.show()

## Analysis of the link between mobility change and democracy index

Load democracy indexes from Our World in Data (https://ourworldindata.org/democracy) :

"Based on the expert assessments and index by V-Dem. It combines information on voting rights, the freedom and fairness of elections,freedoms of association and expression, civil liberties, and executive constraints. It ranges from 0 to 1 (most democratic)"

In [ ]:
# Load the democracy index data
democracy_index = pd.read_csv('data/liberal_democracy_index.csv')

#Extracting the 2020 data
democracy_index_2020 = democracy_index[democracy_index['Year']==2020]

#Drop the useless columns and rename some columns
democracy_index_2020.drop(['Year', 'libdem_vdem_low_owid', 'libdem_vdem_high_owid'], axis=1, inplace=True)
democracy_index_2020.rename(columns={"libdem_vdem_owid": "dem_index"}, inplace=True)

#Drop the data with nan values
democracy_index_2020.dropna(inplace=True)

Enrich the Google and Apple mobility reports with this data by merging the datasets

In [ ]:
#Join with google mobility report
df_google_mob_dem_index = df_google_mobility.merge(democracy_index_2020, left_on='country_region', right_on='Entity')

#Join with Apple mobility report
#Before, change the name of the country Republic of Korea as South Korea so that the merge will work for this country
#df_apple_mobility.loc[df_apple_mobility.region=='Republic of Korea']['region'] = 'South Korea'
df_apple_mobility.replace('Republic of Korea', 'South Korea', inplace=True)
df_apple_mob_dem_index = df_apple_mobility.merge(democracy_index_2020, left_on='region', right_on='Entity')

Compute the mobility drop in the Apple dataset

In [ ]:
#Add columns with the 2 highest and the two lowest weeks in terms of driving/walking
df_apple_mob_dem_index['highest_period_value'] = np.nan
df_apple_mob_dem_index['lowest_period_value'] = np.nan

#Number of days of the period on which the mean is computed
nb_days_period = 14

#Iterate over all countries
for i in range (df_apple_mob_dem_index.shape[0]) :
    # Extract data about the current country
    current_row = df_apple_mob_dem_index.iloc[i]
    country = current_row['region']
    #Keep only the numerical information
    current_row.drop(['geo_type', 'region', 'transportation_type', 'Entity', 'Code', 'dem_index'], inplace=True)
    #Compute the min and max means over 2 floating weeks
    highest_period_value = current_row.rolling(nb_days_period).mean().max()
    lowest_period_value = current_row.rolling(nb_days_period).mean().min()
    #Add these values to the dataset
    df_apple_mob_dem_index['highest_period_value'].iloc[i] = highest_period_value
    df_apple_mob_dem_index['lowest_period_value'].iloc[i] = lowest_period_value

#Considering that the highest mobility value corresponds to the pre lockdown period and the lowest mobility
#corresponds to the lockdown period, we can compute the decrease in the mobility in percents
df_apple_mob_dem_index['mob_variation'] = (df_apple_mob_dem_index['lowest_period_value'] - df_apple_mob_dem_index['highest_period_value']) / df_apple_mob_dem_index['highest_period_value']

Plot the mobility change as a function of the democracy index. Display also the GDP per capita of the different countries

In [ ]:
#Loading the data about GDP per capita
df_gdp = pd.read_csv('data/gdp-per-capita-worldbank.csv')

#Keep only the 2020 values
df_gdp = df_gdp[df_gdp.Year == 2020]

#Rename the GDP column
df_gdp.rename (columns = {'GDP per capita, PPP (constant 2017 international $)' : 'gdp_per_capita'}, inplace=True)

#Drop country code and year columns
df_gdp.drop(['Code', 'Year'], axis=1, inplace=True)

#Add the log of the GDP to the dataframe
df_gdp['log_gdp_per_capita'] = df_gdp['gdp_per_capita'].apply(np.log10)

df_gdp.head()

In [ ]:
# Plotting the mobility change vs democracy index with visualization of the GDP per capita

#Keep only the mobility variations for walking
df_plot = df_apple_mob_dem_index[df_apple_mob_dem_index['transportation_type'] == 'walking']
df_plot = df_plot[['region', 'dem_index', 'mob_variation']].merge(df_gdp, left_on='region', right_on='Entity')


#Plotting the name of the countries
plt.figure(figsize=(16, 7))
for i in range (df_plot.shape[0]) :
    plt.text(x=df_plot.iloc[i]['dem_index']+.006, y=df_plot.iloc[i]['mob_variation']-.01, s=df_plot.iloc[i]['region'], fontdict=dict(color='black',size=7), alpha=.5)

#Scatter plot with the mob_change as a function of the democracy index
plt.scatter(df_plot['dem_index'], df_plot['mob_variation'], c=df_plot['log_gdp_per_capita'], cmap='RdPu')

#Adding titles and showing
plt.title('Decrease in walking mobility with respect to the democracy index')
plt.xlabel('Democracy index')
plt.ylabel('Relative change in walking mobility')
plt.colorbar(label='log of GDP per capita')
#plt.savefig('images/decrease_walking_vs_dem_index.png')

plt.show()

Same thing but with a linear regression on the countries with a democracy index < 0.7

In [ ]:
#Do the same but separating countries which have a dem index lower than .7 and countries > .7

#Keep only the mobility variations for walking
df_plot = df_apple_mob_dem_index[df_apple_mob_dem_index['transportation_type'] == 'walking']
df_plot = df_plot[['region', 'dem_index', 'mob_variation']].merge(df_gdp, left_on='region', right_on='Entity')
df_plot_democratic_countries = df_apple_mob_dem_index[(df_apple_mob_dem_index['transportation_type'] == 'walking') & (df_apple_mob_dem_index['dem_index'] > .7)]
df_plot_democratic_countries = df_plot_democratic_countries[['region', 'dem_index', 'mob_variation']].merge(df_gdp, left_on='region', right_on='Entity')
df_plot_undemocratic_countries = df_apple_mob_dem_index[(df_apple_mob_dem_index['transportation_type'] == 'walking') & (df_apple_mob_dem_index['dem_index'] <= .7)]
df_plot_undemocratic_countries = df_plot_undemocratic_countries[['region', 'dem_index', 'mob_variation']].merge(df_gdp, left_on='region', right_on='Entity')


#Plotting the name of the countries
plt.figure(figsize=(16, 7))
for i in range (df_plot_undemocratic_countries.shape[0]) :
    plt.text(x=df_plot_undemocratic_countries.iloc[i]['dem_index']+.006, y=df_plot_undemocratic_countries.iloc[i]['mob_variation']-.01, s=df_plot_undemocratic_countries.iloc[i]['region'], fontdict=dict(color='black',size=7), alpha=.5)

#Scatter plot with the mob_change as a function of the democracy index
plt.scatter(df_plot['dem_index'], df_plot['mob_variation'], c=df_plot['log_gdp_per_capita'], cmap='RdPu', label='_nolegend_')
plt.colorbar(label='log of GDP per capita')
plt.scatter(df_plot_democratic_countries['dem_index'], df_plot_democratic_countries['mob_variation'], color=[.8, .8, .8], s=70, label='dem_index > 0.7')

#Run linear regression and get coefficients
mod = smf.ols(formula='mob_variation ~ dem_index', data=df_plot_undemocratic_countries)
res = mod.fit()
print (res.summary())
coefficients = res.params.values

#Plot the corresponding line
x_line = np.array([df_plot['dem_index'].min(), 0.7])
y_line = coefficients[0] + coefficients[1] * x_line
plt.plot(x_line, y_line, 'k--', label='linear regression')


#Adding titles and showing
plt.title('Decrease in walking mobility with respect to the democracy index for countries with low democracy index (<0.7)')
plt.xlabel('Democracy index')
plt.ylabel('Relative change in walking mobility')
plt.legend(loc='upper right')
#plt.savefig('images/decrease_walking_vs_dem_index_with_regression.png')

plt.show()

Study the link between GDP per capita and democracy index

In [ ]:
#Now plot the results of the linear regression on the scatter plot

#Getting sns colors
colors = sns.color_palette('deep')

#Scatter plot
plt.figure (figsize=(10, 5))
plt.scatter (df_dem_index_gdp['log_gdp_per_capita'],df_dem_index_gdp['dem_index'], color=colors[7], marker='+', label='Pair GWP/democracy index for different countries')

#Linear regression
x = np.linspace (df_dem_index_gdp['log_gdp_per_capita'].min(), df_dem_index_gdp['log_gdp_per_capita'].max(), 10)
y = coefficients[1]*x + coefficients[0]
plt.plot(x, y, color=colors[5], linestyle='--', label='linear regression')

#Add titles and legend
plt.title("Democracy index as a function of GWP")
plt.xlabel("Log of GDP per capita (2020)")
plt.ylabel('Democracy index')
plt.legend()
#plt.savefig('images/gdp_dem_index.png')

plt.show()

print (res.summary())

### Clustering countries on the typology of their restrictions

For each country, create an associated vector containing the mobility drop for each category. The goal is to cluster countries based on these values to evaluate the restrictions typology and compare it with the democracy index

In [ ]:
#Number of days on which we apply the rolling mean
nb_days_period = 14

#Create an empty dictionnary to store the results
dict_restrictions = {}

#Mobility categories which will be taken into account
categories = ['retail_and_recreation_percent_change_from_baseline', 'grocery_and_pharmacy_percent_change_from_baseline', \
    'transit_stations_percent_change_from_baseline', 'workplaces_percent_change_from_baseline']

#Iterate over all countries
for country in df_google_mob_dem_index['country_region'].unique() :
    #Keep only the values with sub_region_1 is NaN as it corresponds to data for the whole country
    df_country = df_google_mob_dem_index[(df_google_mob_dem_index['country_region'] == country) & (df_google_mob_dem_index['sub_region_1'].isnull())]
    #This list will contain the change between the lowest and the highest period for each category
    changes_list = []

    for category in categories :
        #Extract the data corresponding to this category
        category_data = df_country[category]
        #Compute the highest and the lowest periods
        highest_period_value = category_data.iloc[:30].rolling(nb_days_period).mean().max()
        lowest_period_value = category_data.rolling(nb_days_period).mean().min()
        #Compute the mobility change and add it to the list
        category_change = lowest_period_value - highest_period_value
        changes_list.append(category_change)

    #Add the corresponding entry to the dictionnary
    dict_restrictions[country] = changes_list


Convert the dictinnary dict_restrictions into an array to use in order to use kmeans

In [ ]:
#Import the corresponding modules
from sklearn.cluster import KMeans

#Convert the dictionnary into an array to use it with kmeans
restrictions_array = np.empty((0, len(categories)))
countries = []

#Iterate over all countries
for country in (dict_restrictions.keys()) :
    country_restrictions = np.array(dict_restrictions[country])

    #Check if the row of the corresponding country contains nan values
    if np.isnan(country_restrictions).any() :
        print ('Country containing nan values')
        print (country)

    #If no nan values add it to the array
    else :
        countries.append(country)
        restrictions_array = A = np.vstack([restrictions_array, country_restrictions])


Use the elbow method to evaluate the best number of clusters

In [ ]:
#Define a function which plots the sse

def plot_sse(features_X, start=2, end=11):
    sse = []
    for k in range(start, end):
        # Assign the labels to the clusters
        kmeans = KMeans(n_clusters=k, random_state=10).fit(features_X)
        sse.append({"k": k, "sse": kmeans.inertia_})

    sse = pd.DataFrame(sse)
    # Plot the data
    plt.plot(sse.k, sse.sse)
    plt.xlabel("K")
    plt.ylabel("Sum of Squared Errors")

#Apply it to our data
plot_sse(restrictions_array)



Cluster the data with KMeans, with 5 clusters

In [ ]:
#The best number of clusters is taken equal to 5
n_clusters = 5

#Performing k_means
kmean = KMeans(n_clusters=n_clusters, random_state=2).fit(restrictions_array)


Create a plot summarizing the restrictions for the centroid of each cluster

In [ ]:
#Create a plot summurizing the centroid of each cluster

#Create the figure
plt.figure(figsize=(12, 5))

#Create the categories for the X axis
X = ['Retail & recreation','Grocery & Pharmacy','Transit stations','Workplaces']
X_axis = np.arange(len(X))

#Use seaborn colors
colors = sns.color_palette('deep')

#Create the bar plots
for i in range (kmean.cluster_centers_.shape[0]) :
    plt.bar(X_axis + 0.12*i, kmean.cluster_centers_[i,:], 0.12, label = 'Cluster '+str(i+1), color=colors[i])

#Titles, legends etc
plt.xticks(X_axis+.15*1.5, X)
plt.title("Decrease in the visits to different types of places")
plt.ylabel("Relative decrease (%)")
plt.legend(loc = 'lower right')
#plt.savefig('images/categories_change.png')
plt.show()



Count the number of countries in each bin for democracy index. A bin is for instace [0.3, 0.4[ . The goal of this is to then comput the proportion of countries in each cluster of each bin.

In [ ]:
import math

#Create a list with the democracy indexes of all the countries
all_dem_indexes = []

#Create a list with all the democracy indexes
for country in countries :
    dem_index = df_google_mob_dem_index['dem_index'][df_google_mob_dem_index['country_region']==country].iloc[0]
    all_dem_indexes.append(dem_index)

#Count the number of countries in each bin using the previously created list
countries_dem_index_bins = np.zeros(10)

for i in range (len(all_dem_indexes)) :
    dem_index = all_dem_indexes[i]
    bin = math.floor(dem_index*10)
    countries_dem_index_bins[bin] += 1


#Plot the results
plt.bar([.5+i for i in range(10)], countries_dem_index_bins)
plt.title('Number of countries for each democracy index bin')
plt.xlabel('Democracy index x10')
plt.ylabel('Number of countries')
plt.show()

Display detailed informations for each clusters

In [ ]:
#For each cluster, find the corresponding countries and the mean democracy index

#Getting sns colors for plots
colors = sns.color_palette('deep')

#Iterate over the 5 clusters
for i in range (n_clusters) :

    #Print all the countries which are in the current cluster
    countries_cluster = [] 
    for j in range (kmean.labels_.shape[0]) :
        if kmean.labels_[j] == i :
            countries_cluster.append(countries[j])

    print ('Cluster', i+1, ':')
    print (countries_cluster)

    #Compute the mean and median democracy index of the corresponding cluster
    dem_indexes = []
    for country in countries_cluster :
        dem_index = df_google_mob_dem_index['dem_index'][df_google_mob_dem_index['country_region']==country].iloc[0]
        dem_indexes.append(dem_index)

    mean_dem_index = np.mean(np.array(dem_indexes))
    median_dem_index = np.median(np.array(dem_indexes))

    print ('Mean democracy index :', mean_dem_index)
    print ('Median democracy index :', median_dem_index, '\n')

    #Print the restrictions for the centroid of the cluster
    print ('Restrictions_characteristics')
    for j in range (len(categories)) :
        print (categories[j], kmean.cluster_centers_[i, j], '%')
    
    #For the current cluster, compute for each democracy index bin the proportion of countries contained in the cluster
    cluster_dem_indexes_bins = np.zeros(10)
    for j in range (len(dem_indexes)) :
        dem_index = dem_indexes[j]
        bin = math.floor(dem_index*10)
        cluster_dem_indexes_bins[bin] += 1/ countries_dem_index_bins[bin]
    
    #Plot the corresponding ditribution
    plt.figure(figsize=(6, 4))
    plt.bar([.05+i*.1 for i in range(10)], cluster_dem_indexes_bins, 0.08, color = colors[i])
    plt.xlabel('Democracy index')
    plt.ylabel('Proportion of countries for each bin')
    plt.title('Distribution of the democracy index for cluster ' + str(i+1))
    filename = 'images/cluster_' + str(i+1) + '.png'
    #plt.savefig(filename)
    plt.show()

    print ('\n')
